<a href="https://colab.research.google.com/github/Butanium/tiny-activation-dashboard/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import sys

if 'google.colab' in sys.modules:
  %pip install -qU tiny-dashboard

In [8]:
%load_ext autoreload
%autoreload 2

from tiny_dashboard.feature_centric_dashboards import OfflineFeatureCentricDashboard
from transformers import AutoTokenizer
from collections import defaultdict
from pathlib import Path
import sqlite3
import tempfile
import json
import torch as th

tokenizer = AutoTokenizer.from_pretrained("gpt2")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Offline Dashboard

In [22]:
test_texts = [
    "Hello, how are you?",
    "The quick brown fox jumps over the lazy dog.\n" * 20,
    "I love programming in Python.\n\n" * 100,
    "zaedazzazaaz" * 100,
    "a\na",
]
max_activation_examples: dict[int, list[tuple[float, list[str], list[float]]]] = (
    defaultdict(list)
)
for i in range(0, 100, 10):
    for j, text in enumerate(test_texts):
        toks = tokenizer.tokenize(text)
        acts = [0] * len(toks)
        acts[i % len(acts)] = len(test_texts) - j - 1
        max_activation_examples[i].append((len(test_texts) - j - 1, toks, acts))

dashboards = OfflineFeatureCentricDashboard(max_activation_examples, tokenizer)
dashboards.display()

In [10]:
dashboards.export_to_html("test.html", 10)

### From Database

This should be much faster as we don't load all the examples into memory.

In [11]:
db_path = Path(tempfile.gettempdir()) / "test.db"
if not db_path.exists():
    with sqlite3.connect(db_path) as conn:
        cursor = conn.cursor()
        cursor.execute(
            """CREATE TABLE IF NOT EXISTS data_table (
                key INTEGER PRIMARY KEY,
                examples TEXT
            )"""
        )
        for key, examples in max_activation_examples.items():
            cursor.execute(
                "INSERT INTO data_table (key, examples) VALUES (?, ?)",
                (key, json.dumps(examples)),
            )
        conn.commit()

dashboards = OfflineFeatureCentricDashboard.from_db(
    db_path, tokenizer, column_name="examples"
)
dashboards.display()

use_absolute_max changed to {'name': 'value', 'old': False, 'new': True, 'owner': Checkbox(value=True, description='Use Absolute Max', indent=False, style=DescriptionStyle(description_width='initial')), 'type': 'change'}


## Online Dashboard

In [12]:
from tiny_dashboard.feature_centric_dashboards import (
    AbstractOnlineFeatureCentricDashboard,
)
import torch as th



class DummyOnlineFeatureCentricDashboard(AbstractOnlineFeatureCentricDashboard):
    """Dummy implementation of AbstractOnlineFeatureCentricDashboard"""

    def generate_model_response(self, text: str) -> str:
        return text + "Dummy response"

    def get_feature_activation(
        self, text: str, feature_indicies: tuple[int, ...]
    ) -> th.Tensor:
        activations = []
        for idx in feature_indicies:
            tok_len = len(self.tokenizer.encode(text))
            th.manual_seed(idx)
            activations.append(th.randn(tok_len).exp())
        activations = th.stack(activations, dim=1)
        activations[activations < 3] = 0
        return activations


online_dashboards = DummyOnlineFeatureCentricDashboard(
    AutoTokenizer.from_pretrained("gpt2")
)
online_dashboards.display()

Model is not set, disabling generate response checkbox


If you hate classes, you can use the functions directly

In [13]:
class FakeDict:
    def __getitem__(self, key):
        th.manual_seed(key)
        return th.randn(100).exp().max()
d = FakeDict()
d[3003]

tensor(5.8003)

In [19]:
from tiny_dashboard.feature_centric_dashboards import OnlineFeatureCentricDashboard

class FakeDict:
    def __getitem__(self, key):
        print(f"Getting item for key: {key}")
        th.manual_seed(key)
        v = th.randn(100).exp().max().item() * 2
        print(f"Value: {v}")
        return v

    def __contains__(self, _key):
        return True

def get_feature_activation(
    text: str, feature_indicies: tuple[int, ...]
) -> th.Tensor:
    activations = []
    for idx in feature_indicies:
        tok_len = len(tokenizer.encode(text))
        th.manual_seed(idx)
        activations.append(th.randn(tok_len).exp())
    activations = th.stack(activations, dim=1)
    activations[activations < 3] = 0
    return activations

def generate_model_response(text):
    return text + "Dummy response"


tokenizer = AutoTokenizer.from_pretrained("gpt2")
online_dashboards_2 = OnlineFeatureCentricDashboard(
    get_feature_activation,
    tokenizer,
    generate_model_response,
    max_acts=FakeDict(),
    second_highlight_color=(0, 255, 255),
)
online_dashboards_2.display()

/usr/local/lib/python3.11/dist-packages/tiny_dashboard/feature_centric_dashboards.py:628: UserWarning: Warning:
Model is not set, using DummyModel as a placeholder to allow for response generation using your custom function
  warnings.warn(


## Visualization utils


In [18]:
from tiny_dashboard.visualization_utils import activation_visualization
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokens = tokenizer.tokenize("Hello, how are you?")
primary_activations = th.arange(len(tokens))
secondary_activations = th.arange(len(tokens)).flip(0) * 0.1
# Do not highlight the last token of the secondary activation
secondary_activations[-1] = th.nan

all_activations = th.stack([primary_activations, secondary_activations], dim=1)
print(f"primary_activations: {primary_activations.tolist()}")
print(f"secondary_activations: {secondary_activations.tolist()}")
# Single activation visualization
html_single = activation_visualization(
    tokens=tokens,
    activations=primary_activations,
    tokenizer=tokenizer,
    title="Single Feature",
)
html_single_2 = activation_visualization(
    tokens=tokens,
    activations=secondary_activations,
    tokenizer=tokenizer,
    title="Single Feature",
)

# 2D tensor with highlight feature
html_2d = activation_visualization(
    tokens=tokens,
    activations=all_activations,  # Shape: [num_features, seq_len]
    highlight_idx=0,  # Feature to highlight
    tokenizer=tokenizer,
    title="Feature 0 Highlighted",
)
html_2d_duo = activation_visualization(
    tokens=tokens,
    activations=all_activations,  # Shape: [num_features, seq_len]
    # highlight_idx=[0, 1],  # Feature to highlight
    tokenizer=tokenizer,
    title="Feature 0 and 1 Highlighted",
    color2=(0,255,255)
)
# Rename the features
html_2d_duo_absolute = activation_visualization(
    tokens=tokens,
    activations=all_activations,  # Shape: [num_features, seq_len]
    # highlight_idx=[0, 1],  # Feature to highlight
    tokenizer=tokenizer,
    title="Feature 0 and 1 Highlighted",
    relative_normalization=False,
    activation_names=["Primary", "Secondary"],
    tooltip_features=[0, 1],
)
from IPython.display import HTML

print("Single Feature")
display(HTML(html_single))
print("Single Feature 2")
display(HTML(html_single_2))
print("2D Feature - Only one feature highlighted")
display(HTML(html_2d))
print("2D Feature Duo")
display(HTML(html_2d_duo))
print("2D Feature Duo - Global normalization")
display(HTML(html_2d_duo_absolute))


primary_activations: [0, 1, 2, 3, 4, 5]
secondary_activations: [0.5, 0.4000000059604645, 0.30000001192092896, 0.20000000298023224, 0.10000000149011612, nan]
Single Feature


Single Feature 2


2D Feature - Only one feature highlighted


2D Feature Duo


2D Feature Duo - Global normalization
